In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse


os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg


cfg.N_CLUSTER_RATION = 200

cfg.EDIT_DISTANCE_THRESH_LONG = 0.3
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.3
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 4

cfg.MIN_SUBSTRING_SIZE = 4
cfg.SINGLE_REGEX_SIZE = 4
cfg.TOTAL_REGEX_SIZE = 4
cfg.SINGLE_REGEX_SIZE_RATIO = 0.2
cfg.TOTAL_REGEX_SIZE_RATIO = 0.2

cfg.DOMAIN_CLUSTER_SIZE_THRESH = 3

from file_io import load_urls, dump_urls
from preprocess import make_ip_url_map
from vectorize import make_vectorize
from clustering import make_ip_cluster, make_string_distance_cluster
from regex_url import url_regex_extract, url_regex_check, url_regex_publish
from regex_domain import domain_regex_extract, domain_regex_check
import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
import file_io

In [2]:
urls = load_urls("../../data/VirusTotalnew/malicious/20180416-ma.txt", csv = False)
urls = list(set(urls))
dump_urls(urls, "../../data/EXP_VirusTotal4/malware.csv")
print len(urls)

2018-04-19 19:42:08 file_io.py [line:52] DEBUG	URLs Count:	65667
2018-04-19 19:42:08 file_io.py [line:24] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal4/malware.csv
2018-04-19 19:42:08 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_VirusTotal4/malware.csv


65667


In [3]:
url_domain, url_path = preprocess.make_url_split(urls)
print len(url_domain)
print len(url_path)

13140
56569


In [5]:
url_domain = [ _ for _ in url_domain if not preprocess.check_ip(_)]
print len(url_domain)
urls = list(url_domain)

11292


In [6]:
df_vector = make_vectorize(urls, domain = True, path = False, param = False, dump = False)

2018-04-19 09:13:16 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(11292, 128)


In [7]:
cc_dict = clustering.make_kmeans_cluster(df_vector, cluster_num = 2, dump = False)
cc_list = [cc_dict[_] for _ in cc_dict]

2018-04-19 09:13:19 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 11292 n_clusters: 5
2018-04-19 09:13:19 clustering.py [line:245] DEBUG	optimal n_cluster	2
2018-04-19 09:13:21 clustering.py [line:263] DEBUG	k-means cluster done!


In [9]:
clustering.make_string_distance_cluster(data = cc_list, 
                            metric = "distance",  
                            file_path = "../../data/EXP_VirusTotal4/cluster_distance.json", )

index 0 0/8294
index 0 0/2998
index 400 400/2998
index 600 600/2998
index 800 800/2998
index 1900 1900/2998
index 2300 2300/2998
index 2600 2600/2998
index 200 200/8294
index 400 400/8294
index 500 500/8294
index 600 600/8294
index 700 700/8294
index 900 900/8294
index 1000 1000/8294
index 1200 1200/8294
index 1300 1300/8294
index 1400 1400/8294
index 1700 1700/8294
index 1900 1900/8294
index 2100 2100/8294
index 2200 2200/8294
index 2300 2300/8294
index 2400 2400/8294
index 2800 2800/8294
index 3100 3100/8294
index 3300 3300/8294
index 3400 3400/8294
index 3500 3500/8294
index 3600 3600/8294
index 3700 3700/8294
index 3800 3800/8294
index 4000 4000/8294
index 4100 4100/8294
index 4300 4300/8294
index 4400 4400/8294
index 4500 4500/8294
index 4800 4800/8294
index 5000 5000/8294
index 5200 5200/8294
index 5300 5300/8294
index 5500 5500/8294
index 6400 6400/8294
index 7100 7100/8294
index 7200 7200/8294
index 7300 7300/8294
index 7500 7500/8294
index 7800 7800/8294
index 7900 7900/8294
i

2018-04-19 09:14:08 file_io.py [line:103] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal4/cluster_distance.json
2018-04-19 09:14:08 clustering.py [line:109] DEBUG	string distance time cost:	35.235808


In [19]:
domain_regex_extract?

In [32]:
regex_list = domain_regex_extract(input_file_path = "../../data/EXP_VirusTotal4/cluster_distance.json",
                                 output_file_path = "../../data/EXP_VirusTotal4/regex_raw.txt",
                                 cluster_size_thresh = 4)

2018-04-19 10:44:07 file_io.py [line:172] DEBUG	Cluster Data has been loaded	../../data/EXP_VirusTotal4/cluster_distance.json
2018-04-19 10:44:07 regex_domain.py [line:317] DEBUG	total cluster num:	5356
2018-04-19 10:44:07 regex_domain.py [line:319] DEBUG	big cluster:	200
2018-04-19 10:44:07 regex_domain.py [line:321] DEBUG	small cluster:	732
2018-04-19 10:44:07 regex_domain.py [line:322] DEBUG	single one:	4424
2018-04-19 10:44:07 regex_domain.py [line:323] DEBUG	cluster size detail:	Counter({1: 4424, 2: 531, 3: 133, 4: 68, 5: 43, 6: 30, 7: 21, 8: 18, 9: 7, 14: 7, 11: 6, 10: 5, 12: 5, 13: 5, 16: 4, 21: 4, 15: 3, 17: 3, 19: 2, 29: 2, 42: 2, 48: 2, 58: 2, 61: 2, 20: 1, 22: 1, 150: 1, 25: 1, 26: 1, 2223: 1, 1310: 1, 31: 1, 36: 1, 37: 1, 41: 1, 45: 1, 47: 1, 49: 1, 50: 1, 53: 1, 54: 1, 695: 1, 57: 1, 64: 1, 65: 1, 67: 1, 74: 1, 55: 1, 237: 1, 114: 1, 115: 1})
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'9xiazaiqi'], 2: [u'url'], 3: ['\

2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?dls3|dls2)\.argera\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'com'], 2: [u'duote'], 3: ['\\d{1,2}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^\d{1,2}\.duote\.com\.cn$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'powerpackd', u'powerpackc', u'powerpackmm', u'powerpackdl']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?powerpackd|powerpackc|powerpackmm|powerpackdl)\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'lindevu'], 2: [u'dls4', u'dls1', u'dls3', u'dls2']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?dls4|dls1|dls3|dls2)\.lindevu\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'falezni', u'upaleni'], 2: [u'dls1', u'dls3']})
2018-04-19 10:44:07 regex_domain.py [li

2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'name'], 1: [u'ddns'], 2: ['[A-Za-z]{2,6}tube\\w{3,4}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^[A-Za-z]{2,6}tube\w{3,4}\.ddns\.name$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'iq11download', u'iq6download'], 2: [u'dl2', u'dl5']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?dl2|dl5)\.(:?iq11download|iq6download)\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'dnset'], 2: ['xnxxtube[A-Za-z]{3}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^xnxxtube[A-Za-z]{3}\.dnset\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'dymkilo', u'savtopo', u'kavkito', u'kablits'], 2: [u'dls1', u'dls3']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?dls1|dls3)\.(:?dymkilo|savtopo|kavkito|kablits)\.ru$
201

2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'ibsitva'], 2: [u'dls4', u'dls1', u'dls3', u'dls2']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?dls4|dls1|dls3|dls2)\.ibsitva\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'91mt'], 2: ['\\w{6}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^\w{6}\.91mt\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'falezni', u'benedi', u'balbedi', u'upaleni'], 2: [u'dls4', u'dls1']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?dls4|dls1)\.(:?falezni|benedi|balbedi|upaleni)\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'files-download-67'], 2: [u'mxc', u'4sx']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?mxc|4sx)\.files-download-67\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] D

2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?b2811a66|c3913c6c|853e4f39|5c4e4143)\.(:?xc-webvirusdefence|axc-webvirusdefence)\.nl$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'rcdfh', u'kjdpx', u'yxdfc', u'pc876'], 2: [u'down']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^down\.(:?rcdfh|kjdpx|yxdfc|pc876)\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'rmamos'], 2: [u'dls2']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^dls2\.rmamos\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0755look'], 2: ['\\w{6}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^\w{6}\.0755look\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'aqwitan', u'afdatin'], 2: [u'dls1', u'dls3', u'dls2']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^

2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^dls4\.khemari\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'kr'], 1: [u'co'], 2: ['[A-Za-z]{8}ner'], 3: [u'update']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^update\.[A-Za-z]{8}ner\.co\.kr$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'erotikawyson'], 2: [u'z19415', u'z53053', u'z6690', u'z54596']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:?z19415|z53053|z6690|z54596)\.erotikawyson\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u'itifont'], 2: [u'dls2']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^dls2\.itifont\.ru$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'vid4installer', u'vic6installer', u'v47installer'], 2: [u'dl2', u'dl5']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^(:

2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^13\d{8}\.nuluyu\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'notentop'], 2: ['14\\d{8}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^14\d{8}\.notentop\.com$
2018-04-19 10:44:07 regex_domain.py [line:296] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'ytqihang'], 2: ['14\\d{8}']})
2018-04-19 10:44:07 regex_domain.py [line:297] DEBUG	^14\d{8}\.ytqihang\.com$
2018-04-19 10:44:07 regex_domain.py [line:336] DEBUG	extract regex count:	181
2018-04-19 10:44:07 regex_domain.py [line:340] DEBUG	extract regex count:	174	(after deduplicate)
2018-04-19 10:44:07 regex_domain.py [line:341] DEBUG	extract regex time cost:	0.785236
2018-04-19 10:44:07 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal4/regex_raw.txt
2018-04-19 10:44:07 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal4/regex_raw.txt


In [33]:
df = domain_regex_check(input_file_path = "../../data/EXP_VirusTotal4/regex_raw.txt", 
            test_benign_file_path       = "../../data/sangfor/safe.csv",
            test_malicious_file_path    = "../../data/EXP_VirusTotal4/malware.csv",
            result_file_path            = "../../data/EXP_VirusTotal4/regex_result.txt", 
            n_jobs = 8)

2018-04-19 10:44:11 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal4/regex_raw.txt
2018-04-19 10:44:12 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv
2018-04-19 10:44:12 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/EXP_VirusTotal4/malware.csv


batch index 0	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 13	^2014101\d{10}\.qqldp\.com$
batch index 0	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 15	^\d{1,2}\.duote\.org$
batch index 1	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 19	TP(domain) 4	^down\.(:?rcdfh|kjdpx|yxdfc|pc876)\.com$
batch index 0	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 15	TP(domain) 4	^(:?i1|www|ww17)\.(:?storebox3|storebox1)\.info$
batch index 1	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 1	^dls3\.aparnis\.ru$
batch index 0	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 109	TP(domain) 4	^(:?dl2|dl5)\.(:?vid4installer|vic6installer|v47installer)\.com$
batch index 1	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 22	TP(domain) 4	^(:?dls4|dls1|dls3|dls2)\.igxeweb\.ru$
batch index 0	sample index 4	FP(url) 0	FP(domain) 0	TP(url) 21	TP(domain) 6	^(:?dls1|dls3|dls2)\.(:?aqwitan|afdatin)\.ru$
batch index 2	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 2236	TP(domain) 2236

batch index 3	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 13	^13\d{1,5}92\d{0,5}\.nuluyu\.com$
batch index 7	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 3	^www\.(:?stsweb|leowen)\.(:?net|com)$
batch index 6	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 27	TP(domain) 4	^(:?dls4|dls1|dls3|dls2)\.pxotoxu\.ru$
batch index 5	sample index 6	FP(url) 0	FP(domain) 0	TP(url) 34	TP(domain) 4	^(:?dls1|dls3)\.(:?onigo|opidon)\.ru$
batch index 1	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 1	^dls4\.vapnapi\.ru$
batch index 2	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 10	TP(domain) 4	^(:?www|ww8)\.(:?tw4r|tu8w|wg83)\.com$
batch index 4	sample index 8	FP(url) 0	FP(domain) 0	TP(url) 36	TP(domain) 6	^(:?dls1|dls3)\.(:?dymkilo|savtopo|kavkito|kablits)\.ru$
batch index 0	sample index 17	FP(url) 0	FP(domain) 0	TP(url) 40	TP(domain) 6	^(:?dls4|dls1|dls3)\.(:?repovbi|rezorni)\.ru$
batch index 3	sample index 11	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^1

batch index 3	sample index 20	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^(:?20140829093636314|20140623131417276|20141027141356979|20141026031614616)\.annengdl\.com$
batch index 5	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 1	^dls3\.medyesu\.ru$
batch index 6	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 1	^dl1\.downserver2\.com$
batch index 7	sample index 12	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 9	^update\.vaccine[A-Za-z]{2,7}\.co\.kr$
batch index 4	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 65	TP(domain) 49	^xmilfstube[A-Za-z]{3}\.ddns\.name$
batch index 5	sample index 17	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 3	^www\.winportal\.(:?nl|fr|com)$
batch index 6	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 4	^(:?download11|download18|download14|download10)\.cdn\.(:?compressionfast|compressionfree)\.com$
batch index 7	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 30	TP(domain) 11	^download\d{0,2}\.cdn\.installspeed\.com$
ba

2018-04-19 10:45:55 file_io.py [line:249] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal4/regex_result.txt
2018-04-19 10:45:55 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_VirusTotal4/regex_result.txt


In [2]:
df = regex_domain.domain_regex_publish("../../data/EXP_VirusTotal4/regex_result.txt",
                                  "../../data/EXP_VirusTotal4/regex_publish.txt",
                                  publish_score_thresh = 0.5,
                                  publish_ratio = 10,
                                  publish_fp_thresh = 1000,
                                  publish_tp_thresh = 1)
df = df.loc[df.fp>0]
df

2018-04-20 20:04:34 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_VirusTotal4/regex_result.txt
2018-04-20 20:04:35 regex_domain.py [line:456] DEBUG	regular expression publish	171
2018-04-20 20:04:35 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal4/regex_publish.txt
2018-04-20 20:04:35 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal4/regex_publish.txt


,fp,regex,tp,score,ratio
71,25,^\d{5}\.url\.9xiazaiqi\.com$,21,0.750000,1.190386
72,4,"^\w{4,5}\.downyouxi\.com$",36,0.705882,0.111358
138,1,"^\w{1,4}\.duote\.org$",25,0.666667,0.040384
150,196,^down050\d{5}\.xdown4\.com$,26,0.761905,7.535948
151,149,^\d{5}\.url\.222bz\.com$,82,0.687500,1.816974


In [5]:
regex_list_publish = list(df.regex)
file_io._dump_regex_list(regex_list_publish, "../../data/EXP_VirusTotal4/regex_publish.txt")

2018-04-19 20:09:19 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal4/regex_publish.txt
2018-04-19 20:09:19 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal4/regex_publish.txt


In [6]:
predict_malicious, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = "../../data/sangfor/safe.csv",
                      regex_file_path = "../../data/EXP_VirusTotal4/regex_publish.txt")

2018-04-19 20:09:20 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal4/regex_publish.txt
2018-04-19 20:09:21 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv


batch index 7	sample index 0	hit domain 19	^\d{5}\.url\.9xiazaiqi\.com$
batch index 7	sample index 1	hit domain 4	^\w{4,5}\.downyouxi\.com$
batch index 7	sample index 2	hit domain 1	^\w{1,4}\.duote\.org$
batch index 7	sample index 3	hit domain 158	^down050\d{5}\.xdown4\.com$
batch index 7	sample index 4	hit domain 90	^\d{5}\.url\.222bz\.com$


In [8]:
len(predict_malicious)

375

In [26]:
predict_malicious

['http://down05067522.xdown4.com/cx/160624/35/%E7%8C%8E%E8%B1%B9%E5%85%8D%E8%B4%B9WiFi@894_1965.exe',
 'http://down05072973.xdown4.com/cx/160624/35/%E4%B8%9C%E6%96%B9%E5%85%8D%E8%B4%B9%E5%BD%B1%E8%A7%86%E5%A4%A7%E5%85%A8@319_1_5.exe',
 'http://down05067398.xdown4.com/cx/160624/35/%E9%87%91%E5%B1%B1%E6%89%93%E5%AD%97%E9%80%9A2013@894_2189.exe',
 'http://down05067402.xdown4.com/cx/160624/35/%E5%BE%AE%E4%BF%A1%E7%94%B5%E8%84%91%E7%89%88@894_352.exe',
 'http://down05067388.xdown4.com/cx/160624/35/Excel2007@894_3383.exe',
 'http://down05067388.xdown4.com/cx/160624/35/WPSOffice@894_1318.exe',
 'http://down05067525.xdown4.com/cx/160624/35/Office2010@894_1326.exe',
 'http://down05066006.xdown4.com',
 'http://down05069403.xdown4.com/cx/160624/35/%E4%B8%9C%E6%96%B9%E5%85%8D%E8%B4%B9%E5%BD%B1%E8%A7%86%E5%A4%A7%E5%85%A8@267_1_5.exe',
 'http://down05067478.xdown4.com/cx/160624/35/%E7%BB%9D%E5%9C%B0%E6%B1%82%E7%94%9F%E5%88%BA%E6%BF%80%E6%88%98%E5%9C%BA%E7%94%B5%E8%84%91%E7%89%88@894_3575.exe',
 'htt

# test on sangfor 0410-0416 safe url

In [17]:
predict_malicious, predict_dict, predict_dict_detail = regex_domain.malicious_domain_predict(
                      input_file_path = "../../data/sangfor/malware0416.csv",
                      regex_file_path = "../../data/EXP_VirusTotal4/regex_publish.txt")

2018-04-19 20:14:39 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal4/regex_publish.txt
2018-04-19 20:14:39 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/malware0416.csv


batch index 7	sample index 0	hit domain 4	^\d{5}\.url\.9xiazaiqi\.com$
batch index 7	sample index 1	hit domain 0	^\w{4,5}\.downyouxi\.com$
batch index 7	sample index 2	hit domain 0	^\w{1,4}\.duote\.org$
batch index 7	sample index 3	hit domain 191	^down050\d{5}\.xdown4\.com$
batch index 7	sample index 4	hit domain 0	^\d{5}\.url\.222bz\.com$


In [18]:
len(predict_malicious)

220

In [19]:
for i in predict_dict_detail.keys():
    print i

^down050\d{5}\.xdown4\.com$
^\d{5}\.url\.9xiazaiqi\.com$


In [16]:
939/2075.

0.4525301204819277

In [15]:
for i in predict_dict_detail.keys():
    print i

^(:?down|down3|down2)\.(:?it376|pc876)\.com$
^\w{4,5}\.downyouxi\.com$
^www\.(:?hngp|jsce)\.gov\.cn$
^update\.(:?popupgrade|popupvaccine)\.com$
^\d{5}\.url\.222bz\.com$


In [16]:
predict_dict["^(:?down|down3|down2)\.(:?it376|pc876)\.com$"]

[['http://down2.it376.com',
  'http://down2.it376.com/ico/huoying.ico',
  'http://down2.it376.com/ico/chuanqibaye.ico',
  'http://down2.it376.com/ico/tengxunshipin.ico',
  'http://down2.it376.com/ico/aiqiyi.ico',
  'http://down2.it376.com/ico/baidushurufa.ico',
  'http://down2.it376.com/apk/0612_1.ico']]

In [18]:
predict_dict["^update\.(:?popupgrade|popupvaccine)\.com$"]

[['http://update.popupgrade.com/LMUpdate/Bin/BRmhttp.exe.20180410.html?v=53219444']]

In [13]:
predict_malicious

['http://wt40.downyouxi.com',
 'http://wt40.downyouxi.com/shinydays.zip',
 'http://dx45.downyouxi.com/shinydays.zip',
 'http://dx155.downyouxi.com/xingjidoushiqiyuan.zip?crazycache=1',
 'http://dx40.downyouxi.com/shinydays.zip',
 'http://update.popupgrade.com/LMUpdate/Bin/BRmhttp.exe.20180410.html?v=53219444',
 'http://19929.url.222bz.com/xiaz/Canon%E4%BD%B3%E8%83%BDLBP-2900%E6%BF%80%E5%85%89%E6%89%93%E5%8D%B0%E6%9C%BA%E9%A9%B1%E5%8A%A8R1.14V3.10%E7%89%88ForWin2003-64/Vista-64/2008-64/Win7-64@719_102373.exe',
 'http://18617.url.222bz.com/xiaz/%E6%AC%B2%E6%9C%9B%E4%B9%8B%E8%A1%804%E5%AE%8C%E6%95%B4%E7%89%88@486_208976.exe',
 'http://19934.url.222bz.com/xiaz/HP%E6%83%A0%E6%99%AELaserJet1010%E6%BF%80%E5%85%89%E6%89%93%E5%8D%B0%E6%9C%BA%E6%9C%80%E6%96%B0%E9%A9%B1%E5%8A%A85.60.1604.0%E4%B8%AD%E6%96%87%E7%89%88ForWin98SE/ME/2000/XP@719_87552.exe',
 'http://19931.url.222bz.com/xiaz/%E4%BD%B3%E8%83%BDColorNetworkScanGear%E6%89%AB%E6%8F%8F%E4%BB%AA%E9%A9%B1%E5%8A%A8@373_141200.exe',
 'http://19

In [9]:
predict_malicious, predict_dict, predict_dict_detail = regex_domain.malicious_domain_predict(
                      input_file_path = "../../data/sangfor/malware0416.csv",
                      regex_file_path = "../../data/EXP_VirusTotal4/regex_publish.txt")

2018-04-19 16:21:00 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal4/regex_publish.txt
2018-04-19 16:21:00 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/malware0416.csv


batch index 0	sample index 0	hit domain 0	^2014101\d{10}\.qqldp\.com$
batch index 1	sample index 0	hit domain 0	^down\.(:?rcdfh|kjdpx|yxdfc|pc876)\.com$
batch index 0	sample index 1	hit domain 0	^\d{1,2}\.duote\.org$
batch index 1	sample index 1	hit domain 0	^dls3\.aparnis\.ru$
batch index 1	sample index 2	hit domain 0	^(:?dls4|dls1|dls3|dls2)\.igxeweb\.ru$
batch index 0	sample index 2	hit domain 0	^(:?i1|www|ww17)\.(:?storebox3|storebox1)\.info$
batch index 2	sample index 0	hit domain 0	^\d{10}\.liulanwangye\.com$
batch index 1	sample index 3	hit domain 0	^141630\d{4}\.qiushibaikeba\.com$
batch index 2	sample index 1	hit domain 0	^(:?dls3|dls2)\.argera\.ru$
batch index 0	sample index 3	hit domain 0	^(:?dl2|dl5)\.(:?vid4installer|vic6installer|v47installer)\.com$
batch index 2	sample index 2	hit domain 0	^update\.[A-Za-z]{8}ner\.co\.kr$
batch index 1	sample index 4	hit domain 0	^dls1\.dymkilo\.ru$
batch index 2	sample index 3	hit domain 0	^(:?vd1187|fi1097|fa1857|rs1162)\.91mt\.com$
ba

batch index 6	sample index 0	hit domain 0	^(:?dls4|dls1|dls3|dls2)\.pxotoxu\.ru$
batch index 4	sample index 12	hit domain 0	^\w{7}\.netsolhost\.com$
batch index 5	sample index 4	hit domain 0	^fs05n\d{1}\.sendspace\.com$
batch index 6	sample index 1	hit domain 0	^upload\.(:?jksbn|sjnsdf|jksao|smnck)\.ru$
batch index 4	sample index 13	hit domain 0	^www\.(:?downloadfileshere|download-boosters|download-servers)\.com$
batch index 5	sample index 5	hit domain 0	^download15\.cdn\.archivefast\.com$
batch index 6	sample index 2	hit domain 0	^(:?20140903115704129|20140903111848491|20140903111913567|20140918181111594)\.sltbzy\.com$
batch index 4	sample index 14	hit domain 0	^(:?x|o|dx|cndx)\.uzzf\.com$
batch index 5	sample index 6	hit domain 0	^facebook\.com\.profile\.accounts\.login\.userid41d0\d{4,9}121s513\.(:?4ws5|2ms5)\.com$
batch index 6	sample index 3	hit domain 0	^2014082718\d{7}\.mtvmin\.com$
batch index 4	sample index 15	hit domain 0	^xmilfstube[A-Za-z]{3}\.ddns\.name$
batch index 5	samp

In [10]:
len(predict_malicious)

426

In [11]:
print predict_dict_detail.keys()

['^down050\\d{5}\\.xdown4\\.com$', '^\\w{4,5}\\.downyouxi\\.com$', '^\\d{5}\\.url\\.9xiazaiqi\\.com$', '^\\d{5}\\.url\\.7wkw\\.com$']
